# Prezentacja o bazie NoSQL Qdrant

---

## Autorzy

- Mateusz Malich, Aleksander Batko

---

## Bibliografia

- Oficjalna strona Qdrant: [https://qdrant.tech](https://qdrant.tech)  
- Repozytorium GitHub Qdrant: [https://github.com/qdrant/qdrant](https://github.com/qdrant/qdrant)  
- Artykuły i blogi techniczne o Qdrant (np. Medium, Towards Data Science)  
- Dokumentacja Qdrant: [https://qdrant.tech/documentation/](https://qdrant.tech/documentation/)
- Baza wektorowa: [https://learn.microsoft.com/pl-pl/fabric/real-time-intelligence/vector-database](https://learn.microsoft.com/pl-pl/fabric/real-time-intelligence/vector-database)

---

## Podstawowe informacje o bazie i jej historii

**Qdrant** to nowoczesna baza danych NoSQL typu **vector database**, zaprojektowana do przechowywania i szybkiego wyszukiwania wektorów osadzania (embeddings) generowanych przez modele AI.

### Cechy Qdrant:
- Wysoka wydajność wyszukiwania milionów wektorów przy niskim opóźnieniu  
- Obsługa metadanych do zaawansowanego filtrowania wyników  
- Prosty REST API i gRPC do integracji  
- Możliwość tworzenia i usuwania kolekcji w czasie działania  
- Open Source na licencji Apache 2.0  
- Dostępna lokalnie (on-premises) oraz jako chmurowa usługa Qdrant Cloud  

### Historia:
- Projekt rozpoczęty w 2021 roku przez Qdrant GmbH z Berlina  
- Publiczne wydanie open source w 2022  
- Dynamiczny rozwój w latach 2023-2024 z naciskiem na wydajność i łatwość użycia  

---

### Czym jest baza wektorowa?
Baza wektorowa to specjalny rodzaj bazy danych zaprojektowany do przechowywania i szybkiego wyszukiwania danych w postaci wektorów liczbowych (np. list liczb). Wektory te często powstają jako reprezentacje (embeddingi) obiektów takich jak teksty, obrazy, dźwięki czy inne dane, które dzięki temu można porównywać pod kątem ich znaczenia lub podobieństwa.

## Dlaczego to ważne?
Tradycyjne bazy danych świetnie radzą sobie z danymi strukturalnymi i wyszukiwaniem po kluczach lub pełnotekstowym. Jednak gdy chcemy wyszukiwać np. „podobne obrazy” lub „teksty o podobnym sensie”, potrzebujemy porównywać dane w przestrzeni wielowymiarowej. Do tego służą bazy wektorowe.

## Praktyczny przykład
- Obiekt (np. zdanie) jest zamieniany na wektor liczb (embedding) za pomocą modelu AI
- Wektory umieszczane są w bazie
- Podczas wyszukiwania podajesz zapytanie, które również jest przekształcane na wektor
- Baza szybko znajduje wektory najbardziej „podobne” do zapytania, np. na podstawie odległości kosinusowej lub euklidesowej
---

### Firmy korzystające z Qdrant:

- Microsoft
- Disney dla Disney+
- Kaufland 
- X (Twitter)
- Discord 

## Instalacja i dostępność

### Instalacja lokalna

Na początku trzeba pobrać najnowszy obraz Qdrant z Dockerhuba:

```bash
docker pull qdrant/qdrant
```bash

Potem uruchom jako usługe:

```bash
docker run -p 6333:6333 -p 6334:6334 \
    -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
    qdrant/qdrant
```bash

Następnie inicjalizacja klienta:
```python
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")
```python

Potem trzeba utworzyć kolekcje:
```python
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)
```python

Kolejnym krokiem jest dodanie wektorów:
```python
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)
```python

Następnie mozna zadać proste pytanie: Które z przechowywanych wektorów jest najbardziej podobny do wektora [0.2, 0.1, 0.9, 0.7]?
```python
search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=False,
    limit=3
).points

print(search_result)
```python

W odpowiedzi dostajemy ID najbardziej podobnych wektorów:
```bash
[
  {
    "id": 4,
    "version": 0,
    "score": 1.362,
    "payload": null,
    "vector": null
  },
  {
    "id": 1,
    "version": 0,
    "score": 1.273,
    "payload": null,
    "vector": null
  },
  {
    "id": 3,
    "version": 0,
    "score": 1.208,
    "payload": null,
    "vector": null
  }
]
```bash